# RageSimulator
This simulator is written in rage against how awful the original Piccolissimo simulator is, and rewriting it. 

## Relevant Equations 

### Blade Equations 
$$\gamma = arctan(\frac{[v_E^I]^E [\hat{z}]^E}{[v_E^I]^P [\hat{x}]^E})$$


## Relevant Parameters
* $\nu$ is the induced inflow velocity (Equation 3.1, thesis p.52)
* $[v_E^I]^E$ is the relative wing in the propeller frame.
The origin of the vehicle is unclear, but:
* Center of gravity is relative to somewhere, probably from top of flyer?
* Center of gravity is -2.71e-3 (2.71 mm above body origiin)

## Code parameters

* camelCase unless stated otherwise
* Underscores for frames or for subscripts


In [2]:
%matplotlib inline
import numpy as np 

In [ ]:
# Blade equations 
def getBladeForceMoment(state):
    """Returns forcemoment of a single blade"""
    
    angVelBody_w = state.angVel
    velBody_w = state.vel
    span = 0:dSpan:R_max
    
    # Calculate the relative wind
    rBody2Cg = np.array([0, 0, -2.71e-3]) # Negative z is down
    rBody2CoPStabilizer = -rBody2Cg + np.array([0, 0, -2.56e-3]) # also called h_d or h, CoP is center of Pressure
    heightBody2CoPStabilizer = rBody2CoPStabilizer[2]
    velInf_z = velBody_w[1] + angVelBody_w[1]*heightBody2CoPStabilizer - span * angVelBody_w[2]
    relativeWindAngleGamma = np.atan2(velInf_z, velInf_x)
    